# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     Sara Hernandez Ochoa|   |
| **Fecha**     11/09/2025|   |
| **Expediente**750733 |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm


In [3]:
np.random.seed(42)
n = 100
x = np.random.randn(n)
y = np.random.randn(n)
z = 3 + 5*x + 2*y + np.random.randn(n)

df = pd.DataFrame({"x": x, "y": y, "z": z})


Aquí genero 100 datos aleatorios para x y y, y creo z como una combinación lineal de ellos más algo de ruido, luego los guardo en un DataFrame para poder trabajar con ellos en los modelos de regresión.

In [4]:
X = df[["x", "y"]]
y_var = df["z"]

X_train, X_test, y_train, y_test = train_test_split(X, y_var, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


En este código separa los datos en variables independientes x y dependiente y, luego los divide en conjunto de entrenamiento y prueba (70% y 30%) y finalmente escala las variables para que tengan media 0 y desviación 1, para Ridge y Lasso.

In [5]:
X_sm = sm.add_constant(X_train_scaled)
ols_model = sm.OLS(y_train, X_sm).fit()
print(ols_model.summary())


                            OLS Regression Results                            
Dep. Variable:                      z   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     721.4
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           4.78e-46
Time:                        10:32:45   Log-Likelihood:                -105.51
No. Observations:                  70   AIC:                             217.0
Df Residuals:                      67   BIC:                             223.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.5957      0.133     19.449      0.0

En este modelo OLS se observa un R² de 0.956, lo que indica que el ajuste explica muy bien la variabilidad de la variable dependiente. Los coeficientes estimados para x1 y x2 son aproximadamente 5 y 2, respectivamente, con p-values cercanos a 0, lo que confirma que ambas variables son estadísticamente significativas para predecir z.
z. Además, el intercepto está alrededor de 2.6, alineándose con la forma en que simulamos los datos.

In [6]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)

print("Coeficientes Ridge:", ridge_model.coef_)
print("R2 Ridge:", r2_score(y_test, y_pred_ridge))


Coeficientes Ridge: [4.91310635 1.95216086]
R2 Ridge: 0.9475425906607882


En Ridge los coeficientes bajan a 4.91 y 1.95 por la penalización, y el R² sigue alto en 0.9475, mostrando buen ajuste con parámetros más controlados.

In [7]:
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(X_test_scaled)

print("Coeficientes Lasso:", lasso_model.coef_)
print("R2 Lasso:", r2_score(y_test, y_pred_lasso))


Coeficientes Lasso: [4.87072843 1.87329928]
R2 Lasso: 0.9482664809025805


En Lasso los coeficientes bajan a 4.87 y 1.87 por la penalización L1, y el R² es 0.9483, mostrando buen ajuste con parámetros ligeramente más reducidos.

En general, los tres modelos muestran un buen ajuste de los datos, con R² altos cerca de 0.95. La penalización en Ridge y Lasso reduce ligeramente los coeficientes, ayudando a controlar posibles valores extremos, pero sin afectar significativamente la capacidad predictiva del modelo.